# Implementation of Graph Convolutional Neural Networks (Vanilla)

Based upon:
- **Paper:** Kipf & Welling (ICLR 2017), Semi-Supervised Classification with Graph Convolutional Networks, [https://arxiv.org/pdf/1609.02907.pdf](https://arxiv.org/pdf/1609.02907.pdf)

- **Blog Post:** [https://tkipf.github.io/graph-convolutional-networks/](https://tkipf.github.io/graph-convolutional-networks/)

- **Implementation:** [https://github.com/tkipf/pygcn/tree/master/pygcn](https://github.com/tkipf/pygcn/tree/master/pygcn)

In [1]:
import numpy as np
import scipy.sparse as sp
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import math

## Define Utilities

In [2]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot

In [3]:
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

In [4]:
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

In [5]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32) # Return a COOrdinate representation of this matrix
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [6]:
def load_data(path="./data/cora/", dataset="cora"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0]))

    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test

## GCN Model

In [7]:
class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.sparse.mm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [8]:
class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout
        print(self.gc1)
        print(self.gc2)

    def forward(self, x, adj):
        x = self.gc1(x, adj)
        x = F.relu(x)
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)

## Load Dataset

In [9]:
# Load data
from google.colab import drive
drive.mount('/content/drive')
path = "./drive/MyDrive/Colab Notebooks/Representation Learning for GNNs/data/cora/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
adj, features, labels, idx_train, idx_val, idx_test = load_data(path)

Loading cora dataset...


## Train Model on Cora Dataset

In [11]:
args = {
    "device" : 'cuda' if torch.cuda.is_available() else 'cpu',
    "epochs" : 151,
    "lr" : 0.01,
    "weight_decay" : 5e-4,
    "hidden" : 16,
    "dropout": 0.5
}

In [12]:
# Model and optimizer
model = GCN(nfeat=features.shape[1],
            nhid=args["hidden"],
            nclass=labels.max().item() + 1,
            dropout=args["dropout"])
optimizer = optim.Adam(model.parameters(),
                       lr=args["lr"], weight_decay=args["weight_decay"])

GraphConvolution (1433 -> 16)
GraphConvolution (16 -> 7)


In [13]:
if args["device"] == "cuda":
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()

In [14]:
def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    model.eval()
    output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])

    if epoch % 10 == 0:
      print('Epoch: {:03d}'.format(epoch),
            'loss_train: {:.3f}'.format(loss_train.item()),
            'acc_train: {:.3f}'.format(acc_train.item()),
            'loss_val: {:.3f}'.format(loss_val.item()),
            'acc_val: {:.3f}'.format(acc_val.item()),
            'time: {:.3f}s'.format(time.time() - t))

In [15]:
def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss = {:.3f}".format(loss_test.item()),
          "accuracy = {:.3f}".format(acc_test.item()),
          f" = {round(acc_test.item() * 100, 3)}%")

In [16]:
# Train model
t_total = time.time()
for epoch in range(args["epochs"]):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()

Epoch: 000 loss_train: 1.915 acc_train: 0.114 loss_val: 1.917 acc_val: 0.143 time: 0.032s
Epoch: 010 loss_train: 1.817 acc_train: 0.343 loss_val: 1.816 acc_val: 0.350 time: 0.018s
Epoch: 020 loss_train: 1.748 acc_train: 0.300 loss_val: 1.753 acc_val: 0.350 time: 0.017s
Epoch: 030 loss_train: 1.671 acc_train: 0.357 loss_val: 1.700 acc_val: 0.363 time: 0.018s
Epoch: 040 loss_train: 1.591 acc_train: 0.429 loss_val: 1.633 acc_val: 0.433 time: 0.021s
Epoch: 050 loss_train: 1.450 acc_train: 0.529 loss_val: 1.532 acc_val: 0.490 time: 0.021s
Epoch: 060 loss_train: 1.318 acc_train: 0.600 loss_val: 1.419 acc_val: 0.590 time: 0.017s
Epoch: 070 loss_train: 1.143 acc_train: 0.721 loss_val: 1.296 acc_val: 0.673 time: 0.017s
Epoch: 080 loss_train: 0.999 acc_train: 0.771 loss_val: 1.181 acc_val: 0.733 time: 0.018s
Epoch: 090 loss_train: 0.862 acc_train: 0.793 loss_val: 1.078 acc_val: 0.777 time: 0.017s
Epoch: 100 loss_train: 0.809 acc_train: 0.800 loss_val: 0.986 acc_val: 0.787 time: 0.022s
Epoch: 110